# Part 2.2 

Tested SimpleNN on MNIST from torce 

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

## Using Simple Neural Network & Log_Softmax 

### Optimzer using SGD

In [2]:
import torch.nn.functional as F
import torch.optim as optim

class SNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28,64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,64)
        self.fc4 = nn.Linear(64,10)

    def forward(self,x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x) 
        return F.log_softmax(x, dim=1)

net = SNN()


learning_rate = 0.01 

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Part 2.3
Do write a lot of code snippets to make sure that things are correct.

Leave them here so that we could see that you have tried



In [6]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import glob

# Format of path: root/train/label_name/image-id.jpeg
# Format of path: root/test/label_name/image-id.jpeg


test_directory = "MNIST Dataset JPG format\\MNIST - JPG - testing"
training_directory = "MNIST Dataset JPG format\\MNIST - JPG - training"


full_test_directory = os.path.join(os.getcwd(), test_directory)

full_train_directory = os.path.join(os.getcwd(), training_directory)

subfolders = [f.path for f in os.scandir(full_train_directory) if f.is_dir()]

image_paths = [] # Stores the image paths 


for subfolder in subfolders:
    image_pattern = os.path.join(subfolder, '*.jpg')  # Pattern Matching to get all the JPEG
    image_paths.extend(glob.glob(image_pattern)) # Glob to get all images


class CustomDataset(Dataset): # Since I want to use DataLoader to load batches of images 
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths) # Number of data points 

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
    
        if self.transform:
            image = self.transform(image)  # Apply transformations
    
        # No need to apply ToTensor again, just flatten if required
        image = image.view(-1)  # Flatten the image
    
        label = int(os.path.basename(os.path.dirname(img_path)))  # Extract label
        return image, label

In [ ]:
# simple_cnn_model.to(torch.device('cpu'))

transform = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.Resize((28, 28)),  # Resize 
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])

custom_dataset = CustomDataset(image_paths, transform)
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

losses_over_time = []

# Training loop
num_epochs = 15

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(custom_dataloader):
        optimizer.zero_grad()
        output = net(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    average_loss = running_loss / len(custom_dataloader)

    losses_over_time.append(average_loss) # Append the average loss to the list
    
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss}')

plt.plot(losses_over_time)
plt.title('Training Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

## Testing of the Model with the Test Images

In [ ]:
subfolders = [f.path for f in os.scandir(full_test_directory) if f.is_dir()]
image_paths2 = [] # Stores the image paths 

for subfolder in subfolders:
    image_pattern = os.path.join(subfolder, '*.jpg')  # Pattern Matching to get all the JPEG
    image_paths2.extend(glob.glob(image_pattern)) # Glob to get all images 

# Create the CustomDataset for testing
custom_dataset = CustomDataset(image_paths2, transform)
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Test the model and visualize predictions
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get a batch of test data
dataiter = iter(custom_dataloader)
images, labels = next(dataiter)

# Predict labels
net.eval()  # set to evaluation mode
with torch.no_grad():  # disable gradients (takes memory)
    outputs = simple_cnn_model(images)
_, predicted = torch.max(outputs, 1)

# Show number and prediction side by side
fig = plt.figure(figsize=(25, 4))  # create a figure with a larger size
for idx in range(min(64, batch_size)):  # loop over all images in the batch
    ax = fig.add_subplot(4, 16, idx + 1, xticks=[], yticks=[])  # create a subplot for each image
    ax.imshow(images[idx].squeeze(), cmap='gray')  # show the image in grayscale
    ax.text(
        0.5, -0.5, f'{labels[idx].item()} ({predicted[idx].item()})',
        ha='center',
        color='green' if labels[idx] == predicted[idx] else 'red'
    )  # add text annotation with the number and prediction, using green color for correct predictions and red color for wrong predictions
plt.show()  # show the figure

In [ ]:
# Save the model to .pth
torch.save(net.state_dict(), 'deep_snn_model.pth')

## Dataset and DataLoader
Follow this link https://pytorch.org/tutorials/beginner/basics/data_tutorial.html 
- Define the Dataset Class with 3 functions to get item
- Use the Dataloader in the training loop
- Iterate through the dataloader in the training loop